# Analysis of MovieLens dataset (Beginner'sAnalysis)

Data Source: https://www.kaggle.com/jneupane12/analysis-of-movielens-dataset-beginner-sanalysis
<br>Algorithm: https://towardsdatascience.com/introduction-to-latent-matrix-factorization-recommender-systems-8dfc63b94875
<br>Notebook: https://www.kaggle.com/robottums/probabalistic-matrix-factorization-with-suprise

In [3]:
import sys
print(sys.executable)

E:\ProgramData\Anaconda3\envs\tensorflow_gpuenv\python.exe


In [4]:
import os
os.environ['PATH']

'C:\\Users\\mazha\\AppData\\Roaming\\Python\\Python37\\site-packages\\pywin32_system32;E:\\ProgramData\\Anaconda3\\envs\\tensorflow_gpuenv;E:\\ProgramData\\Anaconda3\\envs\\tensorflow_gpuenv\\Library\\mingw-w64\\bin;E:\\ProgramData\\Anaconda3\\envs\\tensorflow_gpuenv\\Library\\usr\\bin;E:\\ProgramData\\Anaconda3\\envs\\tensorflow_gpuenv\\Library\\bin;E:\\ProgramData\\Anaconda3\\envs\\tensorflow_gpuenv\\Scripts;E:\\ProgramData\\Anaconda3\\envs\\tensorflow_gpuenv\\bin;E:\\ProgramData\\Anaconda3;E:\\ProgramData\\Anaconda3\\Library\\mingw-w64\\bin;E:\\ProgramData\\Anaconda3\\Library\\usr\\bin;E:\\ProgramData\\Anaconda3\\Library\\bin;E:\\ProgramData\\Anaconda3\\Scripts;E:\\ProgramData\\Anaconda3\\bin;E:\\ProgramData\\Anaconda3;E:\\ProgramData\\Anaconda3\\Library\\mingw-w64\\bin;E:\\ProgramData\\Anaconda3\\Library\\usr\\bin;E:\\ProgramData\\Anaconda3\\Library\\bin;E:\\ProgramData\\Anaconda3\\Scripts;E:\\ProgramData\\Anaconda3\\bin;E:\\ProgramData\\Anaconda3;E:\\ProgramData\\Anaconda3\\Librar

## 1. First we import necessary Libaries

In [5]:
import numpy as np #provides numerical arrays and functions to manipulate the arrays efficiently
import pandas as pd # pandas is a data manipulation library
import random
import matplotlib.pyplot as plt # data visualization library
import time
import datetime
import re

## 2.Reading and Exploring the Data

In [6]:
movies = pd.read_csv("datasets/movies.csv")
ratings = pd.read_csv("datasets/ratings.csv")
tags = pd.read_csv("datasets/tags.csv")

### 2.1 Load Movies Data

In [7]:
movies.sample(5)

,movieId,title,genres
7179,72224,Gentlemen Broncos (2009),Comedy
5084,8011,"Weather Underground, The (2002)",Documentary
8400,110297,Muppets Most Wanted (2014),Adventure|Comedy|Crime
2297,3044,Dead Again (1991),Mystery|Romance|Thriller
8750,127323,Vice (2015),Action|Adventure|Sci-Fi|Thriller


In [8]:
movies.shape, movies.info(), len(movies['movieId'].unique().tolist())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


((9742, 3), None, 9742)

### 2.2 Load Ratings Data

In [9]:
ratings.drop_duplicates(inplace=True)
print('we have',ratings.shape[0], 'ratings')
print('the number of unique users we have is:', len(ratings.userId.unique()))
print('the number of unique movies we have is:', len(ratings.movieId.unique()))
print("The median user rated %d books."%ratings.userId.value_counts().median())
print('The max rating is: %d'%ratings.rating.max(),"the min rating is: %d"%ratings.rating.min())

we have 100836 ratings
the number of unique users we have is: 610
the number of unique movies we have is: 9724
The median user rated 70 books.
The max rating is: 5 the min rating is: 0


In [10]:
ratings.sample(5)

,userId,movieId,rating,timestamp
8792,62,6,4.5,1522190219
100368,610,67168,3.5,1493848745
43824,292,70286,2.0,1483193915
38554,265,1225,4.0,965316167
13099,84,105,3.0,858771931


In [11]:
ratings.shape, "\n", ratings.info(), "\n", len(ratings['movieId'].unique().tolist()), "\n", len(ratings['userId'].unique().tolist())
# ratings.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.8 MB


((100836, 4), '\n', None, '\n', 9724, '\n', 610)

### 2.3 Load Tags Data

In [12]:
tags.sample(5)

,userId,movieId,tag,timestamp
435,125,3052,irreverent,1474592072
2134,474,6380,child abuse,1137205738
3397,599,296,big boys with guns,1498456673
3436,599,296,ensemble cast,1498456419
80,62,4223,Ed Harris,1528024869


In [13]:
tags.shape, "\n", tags.info(), len(tags['tag'].unique().tolist())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3683 entries, 0 to 3682
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   userId     3683 non-null   int64 
 1   movieId    3683 non-null   int64 
 2   tag        3683 non-null   object
 3   timestamp  3683 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 115.2+ KB


((3683, 4), '\n', None, 1589)

## 3.Cleaning of data

In [14]:
movies.isnull().any(), " ", ratings.isnull().any(), " ", tags.isnull().any()

# # lets drop null rows
# tags=tags.dropna()

(movieId    False
 title      False
 genres     False
 dtype: bool,
 ' ',
 userId       False
 movieId      False
 rating       False
 timestamp    False
 dtype: bool,
 ' ',
 userId       False
 movieId      False
 tag          False
 timestamp    False
 dtype: bool)

## 4.Data Analysis

### 4.1 Make dates more readable

In [15]:
# https://stackoverflow.com/a/62968313/2049763
def timestamp_to_date_converter(t):
    # https://realpython.com/python-time-module/
    return datetime.datetime.fromtimestamp(t).strftime("%A, %B %d, %Y %I:%M:%S")

In [16]:
ratings['date'] = ratings['timestamp'].apply(timestamp_to_date_converter)
tags['date']    = tags['timestamp'].apply(timestamp_to_date_converter)

In [17]:
ratings.sort_values(by=['timestamp'], ascending=True).tail(5)

,userId,movieId,rating,timestamp,date
81475,514,187031,2.5,1537674927,"Saturday, September 22, 2018 10:55:27"
81477,514,187595,3.0,1537674946,"Saturday, September 22, 2018 10:55:46"
81336,514,5247,2.5,1537757040,"Sunday, September 23, 2018 09:44:00"
81335,514,5246,1.5,1537757059,"Sunday, September 23, 2018 09:44:19"
81092,514,162,4.0,1537799250,"Monday, September 24, 2018 09:27:30"


In [18]:
s = "01/01/2018"
t = time.mktime(datetime.datetime.strptime(s, "%m/%d/%Y").timetuple())

In [19]:
ratings.loc[ratings.timestamp>t].sort_values(by=['timestamp'], ascending=True).head(5)

,userId,movieId,rating,timestamp,date
7181,50,2420,2.5,1514842717,"Monday, January 01, 2018 03:38:37"
7357,50,117529,1.5,1514842744,"Monday, January 01, 2018 03:39:04"
7360,50,122904,1.5,1514891080,"Tuesday, January 02, 2018 05:04:40"
7299,50,72998,2.0,1514891142,"Tuesday, January 02, 2018 05:05:42"
7291,50,63312,2.0,1514891165,"Tuesday, January 02, 2018 05:06:05"


In [20]:
ratings.loc[ratings.timestamp>t].sort_values(by=['timestamp'], ascending=True).shape

(6413, 5)

### 4.2 Reading Movie Release Year

In [21]:
# https://stackoverflow.com/a/8569258
def title_to_release_year(s):
    m = re.findall(r"\(([0-9]+)\)", s)
    
    # https://www.guru99.com/python-regular-expressions-complete-tutorial.html
    if m is None or len(m) <= 0:
        return None        
    return m[-1]

In [22]:
movies['year']  = movies['title'].apply(title_to_release_year)

In [23]:
movies.sample(5)

,movieId,title,genres,year
983,1284,"Big Sleep, The (1946)",Crime|Film-Noir|Mystery,1946
8727,127052,Operation 'Y' & Other Shurik's Adventures (1965),Comedy|Crime|Romance,1965
3013,4031,All the Pretty Horses (2000),Drama|Romance|Western,2000
4356,6373,Bruce Almighty (2003),Comedy|Drama|Fantasy|Romance,2003
2166,2882,Jakob the Liar (1999),Drama,1999


In [24]:
movies.isnull().any()

movieId    False
title      False
genres     False
year        True
dtype: bool

In [25]:
# https://datatofish.com/rows-with-nan-pandas-dataframe/
movies[movies.isnull().any(axis=1)]

,movieId,title,genres,year
6059,40697,Babylon 5,Sci-Fi,None
9031,140956,Ready Player One,Action|Sci-Fi|Thriller,None
9091,143410,Hyena Road,(no genres listed),None
9138,147250,The Adventures of Sherlock Holmes and Doctor W...,(no genres listed),None
9179,149334,Nocturnal Animals,Drama|Thriller,None
9259,156605,Paterson,(no genres listed),None
9367,162414,Moonlight,Drama,None
9448,167570,The OA,(no genres listed),None
9514,171495,Cosmos,(no genres listed),None
9515,171631,Maria Bamford: Old Baby,(no genres listed),None


In [26]:
movies = movies.dropna()

### 4.3 Preparing ratings data

In [27]:
ratings.userId = ratings.userId.astype('category').cat.codes.values
ratings.movieId = ratings.movieId.astype('category').cat.codes.values

ratings.info(), ratings['userId'].value_counts(ascending=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100836 entries, 0 to 100835
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int16  
 1   movieId    100836 non-null  int16  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
 4   date       100836 non-null  object 
dtypes: float64(1), int16(2), int64(1), object(1)
memory usage: 3.5+ MB


(None,
 575      20
 52       20
 430      20
 193      20
 405      20
        ... 
 273    1346
 447    1864
 473    2108
 598    2478
 413    2698
 Name: userId, Length: 610, dtype: int64)

## 5. Basic Recomendation 

https://www.kaggle.com/rajmehra03/cf-based-recsys-by-low-rank-matrix-factorization

In [28]:
# https://stackoverflow.com/a/39881230
# ratings =  ratings.loc[ratings.movieId.isin(movies.movieId)] 

movies_ratings = ratings.merge(movies, on = 'movieId', how = 'inner')
movies_ratings.shape, '# ============== #', ratings.isnull().any()

((67160, 8),
 '# ============== #',
 userId       False
 movieId      False
 rating       False
 timestamp    False
 date         False
 dtype: bool)

In [29]:
movies_ratings = movies_ratings.sort_values(['year', 'timestamp'], ascending=[True, True])
movies_ratings.head(5)

,userId,movieId,rating,timestamp,date,title,genres,year
40402,482,7065,5.0,1256431709,"Saturday, October 24, 2009 07:48:29","Birth of a Nation, The (1915)",Drama|War,1915
40398,380,7065,3.5,1263398689,"Wednesday, January 13, 2010 10:04:49","Birth of a Nation, The (1915)",Drama|War,1915
40401,476,7065,4.0,1268551277,"Sunday, March 14, 2010 01:21:17","Birth of a Nation, The (1915)",Drama|War,1915
40399,413,7065,4.5,1273977081,"Saturday, May 15, 2010 09:31:21","Birth of a Nation, The (1915)",Drama|War,1915
40394,273,7065,4.0,1285018440,"Monday, September 20, 2010 04:34:00","Birth of a Nation, The (1915)",Drama|War,1915


### 5.1 Data loading using Surprise

In [30]:
import surprise

In [31]:
#swapping columns
raw=movies_ratings[['userId','movieId','rating']] 
# when importing from a DF, you only need to specify the scale of the ratings.
reader = surprise.Reader(rating_scale=(1,5)) 
#into surprise:
data_movies_ratings = surprise.Dataset.load_from_df(raw,reader)

### 5.2 Model

In [32]:
class ProbabilisticMatrixFactorization(surprise.AlgoBase):
# Randomly initializes two Matrices, Stochastic Gradient Descent to be able to optimize the best factorization for ratings.
    def __init__(self,learning_rate,num_epochs,num_factors):
       # super(surprise.AlgoBase)
        self.alpha = learning_rate #learning rate for Stochastic Gradient Descent
        self.num_epochs = num_epochs
        self.num_factors = num_factors
    
    def fit(self,train):
        #randomly initialize user/item factors from a Gaussian
        P = np.random.normal(0,.1,(train.n_users,self.num_factors))
        Q = np.random.normal(0,.1,(train.n_items,self.num_factors))
        #print('fit')

        for epoch in range(self.num_epochs):
            for u,i,r_ui in train.all_ratings():
                residual = r_ui - np.dot(P[u],Q[i])
                temp = P[u,:] # we want to update them at the same time, so we make a temporary variable. 
                P[u,:] +=  self.alpha * residual * Q[i]
                Q[i,:] +=  self.alpha * residual * temp 

                
        self.P = P
        self.Q = Q

        self.trainset = train
    
    
    def estimate(self,u,i):
        # returns estimated rating for user u and item i. 
        # Prerequisite: Algorithm must be fit to training set.
        # check to see if u and i are in the train set:
        # print('gahh')

        if self.trainset.knows_user(u) and self.trainset.knows_item(i):
            #print(u,i, '\n','yep:', self.P[u],self.Q[i])
            #return scalar product of P[u] and Q[i]
            nanCheck = np.dot(self.P[u],self.Q[i])
            
            if np.isnan(nanCheck):
                print('Got NaN error, returning the general average')
                return self.trainset.global_mean
            else:
                return np.dot(self.P[u,:],self.Q[i,:])
        else: # if its not known we'll return the general average. 
            print('its not known, returning the general average')
            return self.trainset.global_mean

In [34]:
gs = surprise.model_selection.GridSearchCV(ProbabilisticMatrixFactorization, 
                                           param_grid={'learning_rate':[0.005,0.01],
                                                       'num_epochs':[5,10],
                                                       'num_factors':[10,20]},
                                           measures=['rmse', 'mae'], cv=2)

gs.fit(data_movies_ratings)

In [41]:
print('rsme:', gs.best_score['rmse'],'mae:',gs.best_score['mae'])

best_params = gs.best_params['rmse']
print('best_params \n{\n  rsme: ', gs.best_params['rmse'], '\n  mae:  ', gs.best_params['mae'], " \n}")

rsme: 1.0320173987352135 mae: 0.7880615061323057
best_params 
{
  rsme:  {'learning_rate': 0.01, 'num_epochs': 10, 'num_factors': 10} 
  mae:   {'learning_rate': 0.01, 'num_epochs': 10, 'num_factors': 10}  
}


### 5.2.1 Evaluating the Model Performance

In [36]:
bestVersion = ProbabilisticMatrixFactorization(learning_rate=best_params['learning_rate'],
                                               num_epochs=best_params['num_epochs'],
                                               num_factors=best_params['num_factors'])

In [43]:
#we can use k-fold cross validation to evaluate the best model. 
kSplit = surprise.model_selection.KFold(n_splits=10,shuffle=True)
for train,test in kSplit.split(data_movies_ratings):
    bestVersion.fit(train)
    prediction = bestVersion.test(test)
    surprise.accuracy.rmse(prediction,verbose=True)

RMSE: 0.9515
RMSE: 0.9310
RMSE: 0.9373
RMSE: 0.9368
RMSE: 0.9703
RMSE: 0.9370
RMSE: 0.9329
RMSE: 0.9339
RMSE: 0.9389
RMSE: 0.9642


### 5.2.2 Evaluating the Model Performance

In [45]:
Alg1 = ProbabilisticMatrixFactorization(learning_rate=best_params['learning_rate'],
                                        num_epochs=best_params['num_epochs'],
                                        num_factors=best_params['num_factors'])

data1 = data_movies_ratings.build_full_trainset()
Alg1.fit(data1)

273 7065 : 3.5120607504466945  <--->  4.0


In [48]:
for i in range(10):
    index = random.randint(1, movies_ratings.shape[0] - 1)
    print(index, ": ", movies_ratings.userId.iloc[index], movies_ratings.movieId.iloc[index], 
          movies_ratings.title.iloc[index], ":", 
          round(Alg1.estimate(movies_ratings.userId.iloc[index],movies_ratings.movieId.iloc[index])), 
          " <---> ", round(movies_ratings.rating.iloc[index]))

25503 :  604 5961 Blue Steel (1990) : 4  <--->  2
32283 :  283 379 Timecop (1994) : 2  <--->  4
34065 :  371 361 It Could Happen to You (1994) : 4  <--->  3
37963 :  352 220 Castle Freak (1995) : 3  <--->  3
67059 :  88 8807 Harold and Kumar Go to White Castle (2004) : 4  <--->  5
5713 :  566 8008 Brigadoon (1954) : 4  <--->  2
56668 :  416 3569 Idiots, The (Idioterne) (1998) : 3  <--->  5
65226 :  287 6539 Pirates of the Caribbean: The Curse of the Black Pearl (2003) : 4  <--->  4
28239 :  129 508 Philadelphia (1993) : 3  <--->  3
35463 :  473 735 Cemetery Man (Dellamorte Dellamore) (1994) : 4  <--->  4
